In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score, f1_score

In [31]:
df = pd.read_csv("https://raw.githubusercontent.com/TailUFPB/fundos-imobiliarios/main/preprocessed_df.csv")
df = df.drop(["Período", "Unnamed: 0"], axis=1)
df

,Ativo,Nome,Dividendo,Dividend Yield,Variação Preço,Rentabilidade no Período,Rentabilidade Acumulada,Patrimônio Líquido,Valor Patrimonial,P/VPA,Vacância Física
0,GWIR11,FII GWI RI,0.055168,0.441860,0.870095,0.868693,0.000000,0.025690,214.6766,0.757185,11.10
1,BNFS11,FII BANRISUL,0.032619,0.558140,0.537889,0.560802,0.580113,0.017705,94.7022,0.827051,11.10
2,THRA11B,FII BM THERA,0.000000,0.000000,0.479702,0.458603,0.600784,0.036745,70.4600,0.772803,11.10
3,TRNT11B,FII TORRE NO,0.032080,0.379845,0.529093,0.537516,0.315725,0.318996,167.5480,0.610475,11.10
4,MFII11,FII MERITO I,0.048169,0.860465,0.513532,0.562743,0.409484,0.004704,102.1100,0.699917,11.10
...,...,...,...,...,...,...,...,...,...,...,...
72,RDES11,FII RD ESCRI,0.007979,0.248062,0.422869,0.424968,0.079978,0.020172,95.8878,0.274261,50.53
73,SAAG11,FII SANT AGE,0.033759,0.558140,0.219215,0.255498,0.863145,0.247441,92.0800,0.921803,11.10
74,NSLU11B,FII LOURDES,0.062150,0.620155,0.428958,0.461837,0.469738,0.090507,183.0933,0.691691,11.10
75,AEFI11,FII AESAPAR,0.045287,0.651163,0.479026,0.512290,0.807079,0.034462,131.9325,0.654831,11.10


### Definindo uma Métrica para os FIIs

Vamos definir uma métrica de avaliação que servirá como base para o desempenho do modelo, bem como uma métrica de risco. Tais métricas dependerão de fatores quantitativos relativos à viabilidade do fundo enquanto investimento.

Podemos expressar o risco ou a viabilidade de um investimento ruim (tratando-se de FIIs) como uma combinação linear dos valores P/VPA, Vacância e Dividend Yield, que referem-se, respectivamente, ao quão caro está um fundo, o quanto deste patrimônio está ocioso em movimentações de compra e qual a distribuição de dividendos. Consideraremos, até então, que a viabilidade de um fundo imobiliário é inversamente proporcional aos valores:
  - Vacância
  - P/VPA

e diretamente proporcional aos valores:
  - Rentabilidade
  - Dividend Yield

Logo, temos uma métrica de referência para avaliar, mesmo que intuitivamente, a viabilidade de cada fundo imobiliário, dada por:

M = (Rentabilidade * (Dividend Yield)) / (Vacância * P/VPA)

In [28]:
# for regression
m = df["Rentabilidade no Período"] + df["Dividend Yield"] / (df["P/VPA"] + df["Vacância Física"])

# for classification
c = []
for i in m:
  if (i > 0.6): # crutial, take care when changing it
    c.append(1)
  else:
    c.append(0)
c = pd.Series(c)

0     1
1     1
2     0
3     0
4     1
     ..
72    0
73    0
74    0
75    0
76    1
Length: 77, dtype: int64

In [36]:
X = df.drop(["Ativo", "Nome"], axis=1)
y = m

X_train, X_test, y_train, y_test = train_test_split(X, y)
regressor = RandomForestRegressor().fit(X_train, y_train)
preds = regressor.predict(X_test)

print("R2 Score: ", regressor.score(X_test, y_test))
print("MSE: ", mean_squared_error(y_test, preds))
print("MAE: ", mean_absolute_error(y_test, preds))

R2 Score:  0.8705765296758019
MSE:  0.006779039426033262
MAE:  0.049250662103518396


In [39]:
y = c

X_train, X_test, y_train, y_test = train_test_split(X, y)
classifier = RandomForestClassifier().fit(X_train, y_train)
preds = classifier.predict(X_test)

print("Mean Accuracy: ", classifier.score(X_test, y_test))
print("Precision: ", precision_score(y_test, preds))
print("F1 Score: ", f1_score(y_test, preds))

Mean Accuracy:  0.95
Precision:  1.0
F1 Score:  0.6666666666666666


O modelo explicou bem a variabilidade dos dados, entretanto, uma métrica de avaliação determinada pelo programador deve sempre ser aprimorada. A métrica aqui desenvolvida foi necessária para que fosse possível seguir o paradigma de aprendizado supervisionado; possibilidades de melhoria do modelo incluem técnicas mais avançadas, que podem tornar desnecessária a prévia avaliação humana, ou uma métrica mais precisa e sensível a fatores qualitativos, o que pode tornar o modelo mais realista e robusto.